# Mix the tunes

In [2]:
import numpy as np 
import pandas as pd
import ffmpeg
from pathlib import Path

import glob
import os
import shutil

import IPython.display as ipd
from scipy.io import wavfile
from scipy.signal import resample
import pyrubberband as pyrb


ModuleNotFoundError: No module named 'pyrubberband'

### Pull all the available wavs that you can work with

In [ ]:
def pull_available_wavs(source_dir, target_dir, extension):
    source_dir = os.path.expanduser(source_dir)
    target_dir = os.path.expanduser(target_dir)
    
    # Create target directory if it doesn't exist, make pattern to find wavs via glob match
    os.makedirs(target_dir, exist_ok=True)
    pattern = os.path.join(source_dir, f"*.{extension}")
    matching_files = glob.glob(pattern, recursive=False)

    # Add each file to a list
    available_wavs = [f for f in glob.glob(pattern) if os.path.isfile(f)]
    
    # Copy each file to target directory
    for file_path in matching_files:
        if os.path.isfile(file_path):
            filename = os.path.basename(file_path)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(file_path, target_path)

    return available_wavs

avail_wavs = pull_available_wavs("~/ohw25_proj_RiptideRemix/data/template_sounds", "~/ohw25_proj_RiptideRemix/contributor_folders/isabelle/audio_processing_mixers/available_wavs", "wav")

In [ ]:
print("Playing ", avail_wavs[5])
ipd.Audio(avail_wavs[5])

### Loop file as many times as user requests, but will cut off at 60 seconds

In [ ]:
def loop(input_file, output_file, num_loops, max_duration=60):
    sr, y = wavfile.read(input_file)

    num_samples = y.shape[0]
    max_samples = int(max_duration * sr)

    if y.ndim == 1:
        y_looped = np.tile(y, num_loops)
    else:
        y_looped = np.tile(y, (num_loops, 1))

    y_final = y_looped[:max_samples]

    wavfile.write(output_file, sr, y_final.astype(y.dtype))

    return 

loop(avail_wavs[5], "loop_exp.wav", 3)

In [ ]:
ipd.Audio("loop_exp.wav")

### Increase speed / pitch

In [ ]:
def pitch(input_file, output_file, factor):

    sr, y = wavfile.read(input_file)
    
    num_samples = int(len(y) / factor)
    y_resampled = resample(y, num_samples)
    
    wavfile.write(output_file, sr, y_resampled.astype(y.dtype))
    return

def speed(input_file, output_file, factor):
    sr, y = wavfile.read(input_file)
    y_stretch = pyrb.time_stretch(y, sr, factor)
    wavfile.write(output_file, sr, y_stretch.astype(y.dtype))

    return

speed(avail_wavs[5], "pitch_exp.wav", 1)

In [ ]:
ipd.Audio("pitch_exp.wav")

### Increase amplitude

In [ ]:
def amplitude(input_file, output_file, gain_factor):
    sr, y = wavfile.read(input_file)
    
    y_amplified = y.astype(np.float32) * gain_factor
    
    wavfile.write(output_file, sr, y_amplified)
    return

amplitude(avail_wavs[5], "amp_exp.wav", 1)

In [ ]:
ipd.Audio("amp_exp.wav")

In [ ]:
#fix heightened background noise